# Project imports

In [3]:
"""
All needed imports included here
"""
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import k3d
import trimesh
import torch
import skimage
from torchvision import transforms
import pytorch_lightning as pl



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Data Loading step

In [5]:
"""
Create data loaders and augmentations needed here
"""
from Data.ImageDataLoader import ImageDataLoader
dataloader = ImageDataLoader(25)

# Reconstruction Networks

In [6]:
"""
creation, training, and testing of the image2mesh reconstruction networks
"""
from Networks.Image2Mesh import Image2Voxel

model = Image2Voxel(dataloader,dataloader,dataloader,device)

X = torch.rand(10,3,32,32)
pred = model(X)

torch.Size([10, 1, 32, 32, 32])

In [7]:
trainer = pl.Trainer(
    max_epochs=15,
    gpus=1 if torch.cuda.is_available() else None,
    profiler=True,
    progress_bar_refresh_rate=15
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | Network | 67.1 M


Epoch 0:   0%|          | 0/4 [00:00<?, ?it/s]                        

F:\Python\Anaconda\envs\ml3d\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


Epoch 14: 100%|██████████| 4/4 [00:13<00:00,  3.46s/it, loss=0.483, v_num=16]



Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_fit_start        	|  0.0            	|  0.0            
on_validation_start 	|  0.0019375      	|  0.031          
on_validation_epoch_start	|  0.0            	|  0.0            
on_validation_batch_start	|  0.0            	|  0.0            
validation_step_end 	|  0.0            	|  0.0            
on_validation_batch_end	|  0.0            	|  0.0            
on_validation_epoch_end	|  0.0            	|  0.0            
on_validation_end   	|  5.8144         	|  93.03          
on_train_start      	|  0.016          	|  0.016          
on_epoch_start      	|  0.0010667      	|  0.016          
on_train_epoch_start	|  0.0            	|  0.0            
get_train_batch     	|  0.0037         	|  0.111          
on_batch_start      	|  0.0            	|  0.0            
on_train_batch_start	|  0.0            	|  0.0            
training_ste

1

# Purifying predicted Meshes

In [ ]:
"""
Code to purify meshes predicted by the previous networks to be used in the retrieval step
"""

# Mesh Encoding

In [ ]:
"""
AutoEncoder Models and/or different techniques used to encode the mesh to a smaller dimensions
"""

# Mesh Retreival Networks

In [ ]:
"""
Models/Techniques to use the previous encoding steps to retreive objects from a specified database
"""

# Inference and Full Testing

In [ ]:
"""
Testing the entire pipeline implemented with added visualizations and discussions.
"""

# Citations

[1].....